<a href="https://colab.research.google.com/github/Sirius-Aabhas/CS69002_9A_18CS60R55/blob/master/Execute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import string
from torch.autograd import Variable

##Fetching data for now

In [0]:
df = pd.read_csv('Train_20K.csv', sep='\t')
df.tail()

In [9]:
df_test = pd.read_csv('Test_5K.csv', sep='\t')
df_test.tail()

,text,label
4995,I don't know how to describe this movie. It's ...,0
4996,I found this movie hilarious. The spoofs on ot...,1
4997,My family and I have viewed this movie often o...,1
4998,"The Shining, you know what's weird about this ...",1
4999,Nobody could like this movie for its merit but...,0


In [6]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [7]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]
            
text_reviews = filtered_text_reviews
print(text_reviews[0], text_labels[0])
print(len(text_reviews), len(text_labels))

NameError: ignored

In [0]:
data = [(text_reviews[i], text_labels[i])for i in range(len(text_labels))]
data[:5]

In [10]:
print('Number of Negative movie reviews', len(df_test[df_test['label']==0]))
print('Number of Positive movie reviews', len(df_test[df_test['label']==1]))

Number of Negative movie reviews 2482
Number of Positive movie reviews 2518


In [15]:
text_reviews_test = df_test['text'].astype(str).tolist()
text_labels_test = df_test['label'].astype(int)

text_reviews_test = [x.lower() for x in text_reviews_test]

print(text_reviews_test[0], text_labels_test[0])
print(len(text_reviews_test), len(text_labels_test))

as always this is an inaccurate picture of the homeless. tv told a lot of lies about panhandlers in the early 1990s and made everyone look bad, and claimed we all made over $100 a day when $20-40 a day was much closer to reality. when someone drove by where i held up a sign offering to work, and offered me work, i actually went and took the work if i was physically able.and if i would been offered the $100,000 id damned sure invested in in apt prepaid for at least 2 years, and kept most in the bank and still left myself $10-20000 for nl $1-2 and $2-5 cash games at the casinos. i usually always win and could win decent if i just had a bankroll. instead i win about $1000 a month is all playing in always minimum buying in due to not wanting to risk losing it all. i was only homeless cause i didn't wanna risk spending all my money and going broke, sometimes i had over $1000-2000 in my sock while i slept outside. anyone wanting to talk contact sevencard2003 on yahoo messenger.i admit i was 

In [16]:
data_test = [(text_reviews_test[i], text_labels_test[i])for i in range(len(text_labels_test))]
data_test[:5]

[("as always this is an inaccurate picture of the homeless. tv told a lot of lies about panhandlers in the early 1990s and made everyone look bad, and claimed we all made over $100 a day when $20-40 a day was much closer to reality. when someone drove by where i held up a sign offering to work, and offered me work, i actually went and took the work if i was physically able.and if i would been offered the $100,000 id damned sure invested in in apt prepaid for at least 2 years, and kept most in the bank and still left myself $10-20000 for nl $1-2 and $2-5 cash games at the casinos. i usually always win and could win decent if i just had a bankroll. instead i win about $1000 a month is all playing in always minimum buying in due to not wanting to risk losing it all. i was only homeless cause i didn't wanna risk spending all my money and going broke, sometimes i had over $1000-2000 in my sock while i slept outside. anyone wanting to talk contact sevencard2003 on yahoo messenger.i admit i w

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
    word_to_ix = {}
    word_cntr = {}
    word_set = set()
    #print(dataset)
    for sent in dataset:
        for word in sent.split():
            if word not in word_cntr:
                word_cntr[word] = 1
            else:
                word_cntr[word] += 1
    
    for word in word_cntr:
        if word_cntr[word] >= 5:
            word_to_ix[word] = len(word_to_ix)
            
    word_to_ix['<UNKNOWN>'] = len(word_to_ix)
        
    return word_to_ix

In [25]:
word_to_ix = generate_word_ids(text_reviews + text_reviews_test)
VOCAB_SIZE = len(word_to_ix)
VOCAB_SIZE

NameError: ignored

In [29]:
word_to_ix = np.load('word_dict.npy')
word_to_ix[0]
VOCAB_SIZE

IndexError: ignored

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['<UNKNOWN>']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])


In [0]:
t1 = time.time()
features = make_bow_vector(text_reviews[0], word_to_ix).to(device)
for i in range(1,len(text_reviews[:7000])):
    vec = make_bow_vector(text_reviews[i], word_to_ix).to(device)
    features = torch.cat((features,vec)).to(device)
    if i%1000 == 0:
        print(time.time() - t1)
        t1 = time.time()
        print(features.shape)
targets = torch.tensor(text_labels[:7000], dtype=torch.int).to(device)

print(time.time() - t1)
print(features.shape)
print(targets.shape)
train = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train, batch_size=16, shuffle=True)

In [0]:
def validate(model):
    model = model.cpu()
    preds = []
    for instance, label in data_test:
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        logprobs = model(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        preds.append(pred)

    print('accuracy: {}'.format(accuracy_score(text_labels_test, preds)))
    print('precision: {}'.format(precision_score(text_labels_test, preds)))
    print('recall: {}'.format(recall_score(text_labels_test, preds)))
    print('f1-score: {}'.format(f1_score(text_labels_test, preds)))


In [0]:
mdl = torch.load('Model1A.mdl')

In [0]:
class Model1A(nn.Module):
    def __init__(self, vocab_size):
        super(Model1A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [19]:
validate(mdl)

NameError: ignored